In [ ]:
!pip install spotipy
!pip install requests

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 252.0/252.0 kB 1.7 MB/s eta 0:00:00


In [ ]:
import requests
from spotipy import Spotify
from spotipy.oauth2 import SpotifyClientCredentials
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

# Replace these with your API keys
OPENWEATHERMAP_API_KEY = 'f60a30ad7923bd70572ba9154f6fe83c'
SPOTIPY_CLIENT_ID = '98622a95335f4e6495eb6c6bd50a087b'
SPOTIPY_CLIENT_SECRET = '89cab2ce86d24570bd8d709b7b3f0527'

def get_weather(api_key, city):
    base_url = 'http://api.openweathermap.org/data/2.5/weather'
    params = {'q': city, 'appid': api_key}
    response = requests.get(base_url, params=params)
    data = response.json()
    return data['weather'][0]['main']

def get_spotify_recommendations(client_id, client_secret, seed_genre):
    client_credentials_manager = SpotifyClientCredentials(
        client_id=client_id,
        client_secret=client_secret
    )
    sp = Spotify(client_credentials_manager=client_credentials_manager)
    results = sp.recommendations(seed_genres=[seed_genre], limit=50)  # Increase limit for better representation
    tracks = results['tracks']
    return tracks

def recommend_based_on_weather(weather):
    if 'Rain' in weather or 'Thunderstorm' in weather:
        return 'chill'
    elif 'Snow' in weather:
        return 'classical'
    elif 'Clear' in weather:
        return 'pop'
    else:
        return 'rock'

def recommend_based_on_preference(weather, preference):
    if preference == 'rainy':
        return 'chill'
    elif preference == 'snowy':
        return 'classical'
    elif preference == 'clear':
        return 'pop'
    elif preference == 'cloudy':
        if 'Rain' in weather or 'Thunderstorm' in weather:
            return 'chill'
        else:
            return 'pop'
    else:
        return 'rock'

def recommend_based_on_similarity(preferred_genre, tracks):
    # Extract track names and artists
    track_names = [track['name'] for track in tracks]
    artists = [', '.join([artist['name'] for artist in track['artists']]) for track in tracks]

    # Create text data for TF-IDF vectorizer
    text_data = [name + ' ' + artist for name, artist in zip(track_names, artists)]

    # Vectorize the text data
    vectorizer = TfidfVectorizer()
    tfidf_matrix = vectorizer.fit_transform(text_data)

    # Calculate cosine similarity between track genres and preferred genre
    similarities = cosine_similarity(tfidf_matrix, tfidf_matrix)

    # Find the indices of tracks similar to the preferred genre
    similar_indices = similarities.argsort()[:, -11:-1]  # Top 10 similar tracks

    # Get recommendations from similar tracks
    recommendations = [(track_names[i], artists[i]) for i in similar_indices.flatten()]

    return recommendations

def main():
    city = input('Enter your city: ')
    weather = get_weather(OPENWEATHERMAP_API_KEY, city)
    print(f'The weather in {city} is {weather}')

    preference = input('Enter your weather preference (rainy, snowy, clear, cloudy): ').lower()

    if preference:
        seed_genre = recommend_based_on_preference(weather, preference)
    else:
        seed_genre = recommend_based_on_weather(weather)

    recommendations = get_spotify_recommendations(
        SPOTIPY_CLIENT_ID,
        SPOTIPY_CLIENT_SECRET,
        seed_genre
    )

    recommended_tracks = recommend_based_on_similarity(seed_genre, recommendations)

    print(f'Recommended songs for the weather ({weather}) and preference ({preference}):')
    for i, (song, artist) in enumerate(recommended_tracks, 1):
        print(f'{i}. {song} - {artist}')

if __name__ == '__main__':
    main()


Enter your city: Mathura
The weather in Mathura is Clouds
Enter your weather preference (rainy, snowy, clear, cloudy): Cloudy
Recommended songs for the weather (Clouds) and preference (cloudy):
1. Finest Hour (feat. Abir) - Cash Cash, ABIR
2. Obsession (feat. Jon Bellion) - Vice, Jon Bellion
3. DUELE EL CORAZON (feat. Wisin) - Enrique Iglesias, Wisin
4. Roll in Peace (feat. XXXTENTACION) - Kodak Black, XXXTENTACION
5. Power (feat. Stormzy) - Little Mix, Stormzy
6. LOVE. FEAT. ZACARI. - Kendrick Lamar, Zacari
7. Best Part (feat. H.E.R.) - Daniel Caesar, H.E.R.
8. For Free (feat. Drake) - DJ Khaled, Drake
9. Bitch Better Have My Money - Rihanna
10. Don't - Bryson Tiller
11. All Falls Down (feat. Juliander) - Alan Walker, Noah Cyrus, Digital Farm Animals, Juliander
12. Power (feat. Stormzy) - Little Mix, Stormzy
13. King's Dead (with Kendrick Lamar, Future & James Blake) - Jay Rock, Kendrick Lamar, Future, James Blake
14. Cold Water (feat. Justin Bieber & MØ) - Major Lazer, Justin Bieber,